<a href="https://colab.research.google.com/github/sarath1398/Deep-Learning/blob/main/Face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import google.colab
from google.colab import drive,files

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/

MyDrive  Shareddrives


In [4]:
import os
#print(os.getcwd())
#print(os.listdir(os.getcwd()+'/drive/My Drive'))
os.environ['KAGGLE_CONFIG_PATH']='/content/drive/My Drive/kaggle.json' 
os.environ['KAGGLE_USERNAME'] =  # username from the json file
os.environ['KAGGLE_KEY'] =  # key from the json file
#!kaggle datasets download -d kazanova/sentiment140/

In [5]:
!kaggle datasets download -d jessicali9530/lfw-dataset

 91% 102M/112M [00:01<00:00, 83.8MB/s] 
100% 112M/112M [00:01<00:00, 75.2MB/s]


In [6]:
!unzip lfw-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: lfw-deepfunneled/lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0022.jpg  
  inflating: lfw-deepfunneled/lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0023.jpg  
  inflating: lfw-deepfunneled/lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0024.jpg  
  inflating: lfw-deepfunneled/lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0025.jpg  
  inflating: lfw-deepfunneled/lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0026.jpg  
  inflating: lfw-deepfunneled/lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0027.jpg  
  inflating: lfw-deepfunneled/lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0028.jpg  
  inflating: lfw-deepfunneled/lfw-deepfunneled/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0029.jpg  
  inflating: lfw-deepfunneled/lfw-deepfunneled/Luiz_Inacio_Lula_da_Si

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,Activation,MaxPooling2D
from keras.optimizers import Adam
from keras.initializers import GlorotUniform,Zeros
from keras.losses import binary_crossentropy 

In [9]:
import os

for i in os.listdir(os.getcwd()):
  print(i)

.config
matchpairsDevTest.csv
lfw_readme.csv
lfw-deepfunneled
peopleDevTest.csv
pairs.csv
lfw_allnames.csv
mismatchpairsDevTrain.csv
matchpairsDevTrain.csv
people.csv
mismatchpairsDevTest.csv
lfw-dataset.zip
peopleDevTrain.csv
drive
sample_data


In [10]:
train_df_pos=pd.read_csv('matchpairsDevTrain.csv')
test_df_pos=pd.read_csv('matchpairsDevTest.csv')

train_df_neg=pd.read_csv('mismatchpairsDevTrain.csv')
test_df_neg=pd.read_csv('mismatchpairsDevTest.csv')

In [11]:
train_df_pos.head()

,name,imagenum1,imagenum2
0,Aaron_Peirsol,1,2
1,Aaron_Peirsol,3,4
2,Aaron_Sorkin,1,2
3,Abdel_Nasser_Assidi,1,2
4,Abdullah,1,3


In [12]:
train_df_pos.count()

name         1100
imagenum1    1100
imagenum2    1100
dtype: int64

In [13]:
test_df_pos.head()

,name,imagenum1,imagenum2
0,Abdullah_Gul,13,14
1,Abdullah_Gul,13,16
2,Abdullatif_Sener,1,2
3,Adel_Al-Jubeir,1,3
4,Al_Pacino,1,2


In [14]:
test_df_pos.count()

name         500
imagenum1    500
imagenum2    500
dtype: int64

In [15]:
train_df_neg.head()

,name,imagenum1,name.1,imagenum2
0,AJ_Cook,1,Marsha_Thomason,1
1,Aaron_Sorkin,2,Frank_Solich,5
2,Abdel_Nasser_Assidi,2,Hilary_McKay,1
3,Abdoulaye_Wade,4,Linda_Dano,1
4,Abdul_Rahman,1,Magui_Serna,1


In [16]:
train_df_neg.count()

name         1100
imagenum1    1100
name.1       1100
imagenum2    1100
dtype: int64

In [17]:
test_df_neg.count()

name         500
imagenum1    500
name.1       500
imagenum2    500
dtype: int64

In [18]:
test_df_neg.head()

,name,imagenum1,name.1,imagenum2
0,AJ_Lamas,1,Zach_Safrin,1
1,Aaron_Guiel,1,Reese_Witherspoon,3
2,Aaron_Tippin,1,Jose_Luis_Rodriguez_Zapatero,1
3,Abdul_Majeed_Shobokshi,1,Charles_Cope,1
4,Abdullah_Gul,16,Steve_Cox,1


In [19]:
print(len(os.listdir(os.getcwd()+'/lfw-deepfunneled/lfw-deepfunneled')))

5749


In [20]:
from PIL import Image

celeb=sorted(os.listdir(os.getcwd()+'/lfw-deepfunneled/lfw-deepfunneled'))[0]
print(celeb)

AJ_Cook


In [21]:
del celeb

In [22]:
%matplotlib inline

path=os.getcwd()+'/lfw-deepfunneled/lfw-deepfunneled/'
image_di={}

for celeb in os.listdir(path):

  image_di[celeb]=[]

  for image in os.listdir(path+'/'+celeb):

    abspath=path+'/'+celeb+'/'+image
    img=Image.open(abspath)
    img1=img.copy()
    img1.thumbnail((224,224))
    image_di[celeb].append(img1)

    #img1.save(abspath)


In [23]:
orig_path=os.getcwd()
os.mkdir('All_Images')

In [24]:
%%time

os.chdir(os.getcwd()+'/All_Images')
for celeb in sorted(image_di.keys()):

  count=1
  for obj in image_di[celeb]:

    obj.save('{}_{}.jpg'.format(celeb,count))
    count+=1

CPU times: user 24.3 s, sys: 809 ms, total: 25.1 s
Wall time: 25.1 s


In [25]:
del image_di

In [26]:
os.chdir(orig_path)

for i in os.listdir(os.getcwd()+'/All_Images')[:5]:
  print(i)

Franck_Cerutti_1.jpg
Kathryn_Tucker_1.jpg
Justin_Leonard_3.jpg
Tony_Blair_21.jpg
Joschka_Fischer_3.jpg


**Create Match Pairs**

In [27]:
%%time

match_test_pairs_images=[]
match_train_pairs_images=[]

for i in range(train_df_pos['name'].count()):

  name=train_df_pos['name'][i]
  ino_1=train_df_pos['imagenum1'][i]
  ino_2=train_df_pos['imagenum2'][i]

  image_1=name+'_'+str(ino_1)+'.jpg'
  image_2=name+'_'+str(ino_2)+'.jpg'

  match_train_pairs_images.append([image_1,image_2])

for i in range(test_df_pos['name'].count()):

  name=test_df_pos['name'][i]
  ino_1=test_df_pos['imagenum1'][i]
  ino_2=test_df_pos['imagenum2'][i]

  image_1=name+'_'+str(ino_1)+'.jpg'
  image_2=name+'_'+str(ino_2)+'.jpg'

  match_test_pairs_images.append([image_1,image_2])

print(match_train_pairs_images[:5])
print(match_test_pairs_images[:5])

[['Aaron_Peirsol_1.jpg', 'Aaron_Peirsol_2.jpg'], ['Aaron_Peirsol_3.jpg', 'Aaron_Peirsol_4.jpg'], ['Aaron_Sorkin_1.jpg', 'Aaron_Sorkin_2.jpg'], ['Abdel_Nasser_Assidi_1.jpg', 'Abdel_Nasser_Assidi_2.jpg'], ['Abdullah_1.jpg', 'Abdullah_3.jpg']]
[['Abdullah_Gul_13.jpg', 'Abdullah_Gul_14.jpg'], ['Abdullah_Gul_13.jpg', 'Abdullah_Gul_16.jpg'], ['Abdullatif_Sener_1.jpg', 'Abdullatif_Sener_2.jpg'], ['Adel_Al-Jubeir_1.jpg', 'Adel_Al-Jubeir_3.jpg'], ['Al_Pacino_1.jpg', 'Al_Pacino_2.jpg']]
CPU times: user 36.6 ms, sys: 0 ns, total: 36.6 ms
Wall time: 36.1 ms


In [28]:
%%time

os.chdir(orig_path+'/All_Images/')

match_train_pairs=[]
match_test_pairs=[]

match_train_labels=[]

for pairs in match_train_pairs_images:

  img_name_1,img_name_2=pairs

  for img in os.listdir(os.getcwd()):

    if img_name_1.lower()==img.lower():
      img_obj_1=Image.open(img_name_1)
      break
  
  for img in os.listdir(os.getcwd()):

    if img_name_2.lower()==img.lower():
      img_obj_2=Image.open(img_name_2)
      break
  
  match_train_pairs.append([img_obj_1,img_obj_2])
  match_train_labels.append(1)

CPU times: user 9.79 s, sys: 9.44 s, total: 19.2 s
Wall time: 19.3 s


In [29]:
match_train_pairs[:5]

[[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35358>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA351D0>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35240>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA354A8>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35208>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35080>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35E48>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35198>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35128>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35780>]]

In [30]:
match_test_labels=[]

for pairs in match_test_pairs_images:

  img_name_1,img_name_2=pairs

  for img in os.listdir(os.getcwd()):

    if img_name_1.lower()==img.lower():
      img_obj_1=Image.open(img_name_1)
      break
  
  for img in os.listdir(os.getcwd()):

    if img_name_2.lower()==img.lower():
      img_obj_2=Image.open(img_name_2)
      break
  
  match_test_pairs.append([img_obj_1,img_obj_2])
  match_test_labels.append(1)

In [31]:
match_test_pairs[:5]

[[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA355C0>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35630>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35668>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35518>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA356D8>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35470>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35588>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35390>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35828>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0CA35160>]]

**Mismatch Image Pairs**

In [32]:
%%time

mismatch_test_pairs_images=[]
mismatch_train_pairs_images=[]

mismatch_test_labels=[]
mismatch_train_labels=[]

for i in range(train_df_neg['name'].count()):

  name1=train_df_neg['name'][i]
  name2=train_df_neg['name.1'][i]

  ino_1=train_df_neg['imagenum1'][i]
  ino_2=train_df_neg['imagenum2'][i]

  image_1=name1+'_'+str(ino_1)+'.jpg'
  image_2=name2+'_'+str(ino_2)+'.jpg'

  mismatch_train_pairs_images.append([image_1,image_2])
  mismatch_train_labels.append(0)

for i in range(test_df_neg['name'].count()):

  name1=test_df_neg['name'][i]
  name2=test_df_neg['name.1'][i]
  ino_1=test_df_neg['imagenum1'][i]
  ino_2=test_df_neg['imagenum2'][i]

  image_1=name1+'_'+str(ino_1)+'.jpg'
  image_2=name2+'_'+str(ino_2)+'.jpg'

  mismatch_test_pairs_images.append([image_1,image_2])
  mismatch_test_labels.append(0)

print(mismatch_train_pairs_images[:5])
print(mismatch_test_pairs_images[:5])

[['AJ_Cook_1.jpg', 'Marsha_Thomason_1.jpg'], ['Aaron_Sorkin_2.jpg', 'Frank_Solich_5.jpg'], ['Abdel_Nasser_Assidi_2.jpg', 'Hilary_McKay_1.jpg'], ['Abdoulaye_Wade_4.jpg', 'Linda_Dano_1.jpg'], ['Abdul_Rahman_1.jpg', 'Magui_Serna_1.jpg']]
[['AJ_Lamas_1.jpg', 'Zach_Safrin_1.jpg'], ['Aaron_Guiel_1.jpg', 'Reese_Witherspoon_3.jpg'], ['Aaron_Tippin_1.jpg', 'Jose_Luis_Rodriguez_Zapatero_1.jpg'], ['Abdul_Majeed_Shobokshi_1.jpg', 'Charles_Cope_1.jpg'], ['Abdullah_Gul_16.jpg', 'Steve_Cox_1.jpg']]
CPU times: user 53.9 ms, sys: 48 µs, total: 53.9 ms
Wall time: 55.4 ms


In [33]:
mismatch_train_pairs=[]
mismatch_test_pairs=[]

for pairs in mismatch_train_pairs_images:

  img_name_1,img_name_2=pairs

  for img in os.listdir(os.getcwd()):

    if img_name_1.lower()==img.lower():
      img_obj_1=Image.open(img_name_1)
      break
  
  for img in os.listdir(os.getcwd()):

    if img_name_2.lower()==img.lower():
      img_obj_2=Image.open(img_name_2)
      break
  
  mismatch_train_pairs.append([img_obj_1,img_obj_2])

In [34]:
mismatch_train_pairs[:5]

[[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0AAC7470>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0AAC75C0>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0AAC77F0>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0AAC7748>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0AAC7780>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0AAC7710>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0AAC75F8>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0AAC7860>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0AAC7940>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0AAC7898>]]

In [35]:
for pairs in mismatch_test_pairs_images:

  img_name_1,img_name_2=pairs

  for img in os.listdir(os.getcwd()):

    if img_name_1.lower()==img.lower():
      img_obj_1=Image.open(img_name_1)
      break
  
  for img in os.listdir(os.getcwd()):

    if img_name_2.lower()==img.lower():
      img_obj_2=Image.open(img_name_2)
      break
  
  mismatch_test_pairs.append([img_obj_1,img_obj_2])

In [36]:
mismatch_test_pairs[:5]

[[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F09BDFA20>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F09BDFA90>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F09BDFB00>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F09BDFB70>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F09BDFBE0>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F09BDFC50>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F09BDFCC0>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F09BDFD30>],
 [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F0BBC74E0>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F0F09BDFDA0>]]

In [37]:
train_pairs=match_train_pairs+mismatch_train_pairs
test_pairs=match_test_pairs+mismatch_test_pairs

In [38]:
train_labels=np.asarray(match_train_labels+mismatch_train_labels,dtype='float32')
test_labels=np.asarray(match_test_labels+mismatch_test_labels,dtype='float32')

In [39]:
del match_train_pairs_images
del match_test_pairs_images
del mismatch_train_pairs_images
del mismatch_test_pairs_images

os.chdir(orig_path)
#os.remove('/lfw-deepfunneled')

In [40]:
sample_obj=train_pairs[0][0]
sample_nparr=np.asarray(sample_obj)
sample_nparr.shape

(224, 224, 3)

In [41]:
train_numpy_pairs=[]
test_numpy_pairs=[]

for pairs in train_pairs:
  pair=[]
  for image in pairs:
    pair.append(np.asarray(image,dtype='float32')/255)
  train_numpy_pairs.append(np.asarray(pair,dtype='float32'))

for pairs in test_pairs:
  pair=[]
  for image in pairs:
    pair.append(np.asarray(image,dtype='float32')/255)
  test_numpy_pairs.append(np.asarray(pair,dtype='float32'))


In [42]:
train_nparr_pairs=np.asarray(train_numpy_pairs)

In [43]:
del train_numpy_pairs

In [44]:
test_nparr_pairs=np.asarray(test_numpy_pairs)
del test_numpy_pairs

In [45]:
train_labels[1096:1110]

array([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [46]:
from keras.layers import Input,Lambda,BatchNormalization,Dropout
from keras import backend as K
from keras.models import Model
from keras.regularizers import l2
from keras.applications.inception_v3 import InceptionV3

IMG_SHAPE=(128,128,3)
BATCH_SIZE=16
EPOCHS=32

def return_siamese_net():

  left_input=Input(IMG_SHAPE)
  right_input=Input(IMG_SHAPE)

  model=Sequential()#name="VGG-16")

  #First Layer

  model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same',input_shape=IMG_SHAPE,kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-4)))
  model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-4)))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  #Second Layer
  model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-4)))
  model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-4)))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  '''
  #Third Layer
  model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-4)))
  model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-4)))
  model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-4)))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  #Fourth Layer
  model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-4)))
  model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-4)))
  model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-4)))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  #Fifth Layer
  model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-4)))
  model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-4)))
  model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-4)))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
  '''
  #Sixth Layer
  model.add(Flatten())
  model.add(Dense(4096, activation='relu'))
  model.add(Dense(4096,activation='relu'))
  model.add(Dropout(0.3))
  '''
  model=InceptionV3(include_top=False,weights='imagenet',input_shape=IMG_SHAPE)
  model.add(Flatten())
  model.add(Dense(4096,activation='relu'))
  model.add(Dense(4096,activation='relu'))
  '''
  model.add(Dense(2,activation='sigmoid'))

  encoded_l=model(left_input)
  encoded_r=model(right_input)

  lambda_layer= Lambda(lambda tensors:K.abs(tensors[0]-tensors[1]))
  L1_distance = lambda_layer([encoded_l, encoded_r])
  prediction = Dense(2,activation='sigmoid')(L1_distance)
  siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)  
  return siamese_net

In [47]:
model=return_siamese_net()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 2)            8819266     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 2)            0           sequential[0][0]             

In [48]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

In [49]:
from keras.optimizers import SGD,RMSprop,Adam

optimizer=Adam(lr=0.01)
model.compile(loss=contrastive_loss,metrics=['accuracy'],optimizer=optimizer)

In [50]:
#train_nparr_pairs=np.reshape(train_nparr_pairs,(2200,2,224*224*3))
#train_nparr_pairs.shape

In [51]:
#test_labels=np.expand_dims(test_labels,axis=-1)
#test_labels.shape

In [ ]:
model.fit(x=[[train_nparr_pairs[:, 0], train_nparr_pairs[:, 1]]], y=train_labels[:], 
          validation_data=([[test_nparr_pairs[:, 0], test_nparr_pairs[:, 1]]], test_labels[:]), 
          batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/32
  4/138 [..............................] - ETA: 1:55:08 - loss: 0.5372 - accuracy: 0.4701

In [ ]:
model.save('face_recognizer.h5')